# Objetivos
- Sacar datos gasolina por mes (2019-2022) : https://energia.serviciosmin.gob.es/shpCarburantes/ (SELENIUM)
    - Lo vamos a sacar de Gasolina 95,98,Diesel y Diesel Premium

- Sacar datos Agua:
    - https://facua.org/es/tablas/agua2019.pdf
    - https://facua.org/es/tablas/agua_2020.pdf
    - https://facua.org/es/tablas/agua2021.pdf
    - https://facua.org/es/tablas/agua2022.pdf

In [ ]:
# Importamos las librerías que necesitamos

# Librerías para control y automatización de navegadores (Web Scraping)
# -----------------------------------------------------------------------
from selenium import webdriver                   # Para controlar el navegador automáticamente
from selenium.webdriver.chrome.service import Service  # Para manejar los servicios de ChromeDriver
from selenium.webdriver.common.by import By      # Para seleccionar elementos en el DOM de la página
from selenium.webdriver.chrome.options import Options  # Para establecer opciones en el navegador (como headless mode)

# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipular estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y arrays multidimensionales
import datetime                 # Para manejar fechas y horas

In [ ]:
fechas_inicio_mes = pd.date_range(start='2019-01-01', end='2022-12-01', freq='MS')
fechas_inicio_mes

In [ ]:
fechas_fin_mes = pd.date_range(start='2019-01-31', end='2022-12-31', freq='ME')
fechas_fin_mes